# Concise Models
Created by Christoffer Rubensson on the 2025-11-29.

This notebook will soon be filled with examples on how to create and evaluate concise models.

*To be continue.*

In [ ]:
#TODO

### Initialization

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
# App directory
project_root = Path.cwd().parent  # Adjust this if necessary
sys.path.append(str(project_root))

**Import packages**

*For import without pip*

In [3]:
from src.variant_extraction.utils.data_importing import load_event_log

In [4]:
log = load_event_log("bpic2013_incidents.xes", "input")

/Users/christofferrubensson/Documents/GITHUB/VARIANT_EXTRACTION/data/input/bpic2013_incidents.xes


/Users/christofferrubensson/Documents/GITHUB/VARIANT_EXTRACTION/concvenv/lib/python3.11/site-packages/pm4py/utils.py:992: UserWarning: Install the optional requirement `rustxes` to import/export files faster.
  warnings.warn("Install the optional requirement `rustxes` to import/export files faster.")
/Users/christofferrubensson/Documents/GITHUB/VARIANT_EXTRACTION/concvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 7554/7554 [00:02<00:00, 3252.04it/s]
